In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_nodes, export_model_as_feature
from train import run_experiment

2022-01-10 00:34:25.721767: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load audience overlap edges for level 1

In [2]:
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2020_audience_overlap_sites_scrapping_result.json'), level=1)
audience_overlap_sites_NODES = create_nodes(audience_overlap_sites)

print(audience_overlap_sites_NODES[:5])

01-10 00:34:27 notebooks.utils INFO     Loaded 3489 nodes with records level <= 1 and child size:16981


[('crooked.com', 'votesaveamerica.com'), ('crooked.com', 'art19.com'), ('crooked.com', 'promocodeportal.com'), ('crooked.com', 'mediamatters.org'), ('crooked.com', 'actblue.com')]


In [3]:
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,crooked.com,votesaveamerica.com
1,crooked.com,art19.com
2,crooked.com,promocodeportal.com
3,crooked.com,mediamatters.org
4,crooked.com,actblue.com


### Find all unique nodes in edges

In [4]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 10161 Sample: ['q13fox.com', 'influencewatch.org', 'dennisdemori.com', 'fiercehealthcare.com', 'jfranews.com.jo']


### 1. Load all node features

In [5]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [6]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [7]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10161 entries, q13fox.com to carefree.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   7465 non-null   float64
 1   daily_pageviews_per_visitor  7466 non-null   float64
 2   daily_time_on_site           5566 non-null   float64
 3   total_sites_linking_in       9861 non-null   float64
 4   bounce_rate                  5179 non-null   float64
dtypes: float64(5)
memory usage: 476.3+ KB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [8]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10161 entries, q13fox.com to carefree.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   10161 non-null  float64
 1   daily_pageviews_per_visitor  7466 non-null   float64
 2   daily_time_on_site           5566 non-null   float64
 3   total_sites_linking_in       10161 non-null  float64
 4   bounce_rate                  5179 non-null   float64
dtypes: float64(5)
memory usage: 476.3+ KB


### 3. Normalizing features

In [9]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [10]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 10161, Edges: 17010

 Node types:
  default: [10161]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [17010]
        Weights: all 1 (default)
        Features: none


# Unsupervised GraphSAGE
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

In [11]:
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 128
epochs = 4
num_samples = [10, 5]

graphsage_link_generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_graphsage_link_gen = graphsage_link_generator.flow(unsupervised_samples)

layer_sizes = [128, 512]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=graphsage_link_generator, bias=True, dropout=0.0, normalize="l2"
)

# Build the model and expose input and output sockets of graphsage, for node pair inputs:
x_inp, x_out = graphsage.in_out_tensors()

prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

graphsage_model = keras.Model(inputs=x_inp, outputs=prediction)

graphsage_model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.categorical_crossentropy,
    metrics=['acc'],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [13]:
history = graphsage_model.fit(
    train_graphsage_link_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

"""
Epoch 1/4
   2/5047 [..............................] - ETA: 3:09 - loss: 0.7847 - binary_accuracy: 0.515 - ETA: 9:20 - loss: 0.7903 - binary_accuracy: 0.5039WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0129s vs `on_train_batch_end` time: 0.2059s). Check your callbacks.
5047/5047 [==============================] - 548s 109ms/step - loss: 0.5881 - binary_accuracy: 0.7075 8:59 - ETA: 8:3 - ETA: 8:01 - - ETA: 7:50 - loss: 0.6312  - ETA: 7:35 - loss: 0.6251 - bin - ETA: 7:13 - loss: 0.6196 - binary_acc - ETA: 7:11 - loss: 0.6192 -  - ETA: 7:08 - loss: 0.6188 -  - ETA: 7:04 - loss: 0.6183 - binary_accu - ETA: 7:04 - loss: 0.6179 - binary_a - ETA: 7:01 - loss: 0.6174 -  - ETA: 6:52 - los - ETA: 6:48 - loss: 0.6148 - binary_accuracy: 0. - ETA: 6:47 - loss: 0.6146 - - ETA: 6:45 - loss: 0.6 - ETA: 6:41  - ETA: 6:31 - loss: 0.6121 - binary_accuracy: 0.6 - ETA: 6:30 - loss: 0. - ETA: 6:26 - los - ETA: 6:02 - loss: 0.608 - ETA: 5:58 -  - ETA: 5:54  - ETA: 4:32 - loss: 0.6004 -  - ETA: 4:30 - loss: 0.6001 - binary_accuracy:  - ETA: 4:29 - loss: 0.6001 - binary_accuracy: - ETA: 4:28 - loss: 0.6001 - binary_accu - ETA: 10s - loss: 0.5883 - bina - ETA: 9s - loss: 0.5883 - binary_accuracy: 0.7 - ETA: 9s - loss: 0.5883 - binary 
Epoch 2/4
5047/5047 [==============================] - 547s 108ms/step - loss: 0.5730 - binary_accuracy: 0.7293 9:22 - loss: 0.5795 - binary_accur - ETA: 8:40 - loss: 0.5779 - binary_accuracy - ETA: 8:38 - loss: 0.5780 - binary_accuracy: 0. - ETA: 8:37 - loss: 0.5778 - binary_accuracy - ETA: 8:36 - loss: 0.5779 - b - ETA:  - ETA: 5:54 - loss: 0.57 - ETA: 5:31 - lo - ETA: 5:02 - loss: 0.57 - ETA: 4:58 - loss: 0.5736 - binary_accuracy: 0.7 - ETA: 4:58 - loss: 0.5737 - - ETA: 4:48 - loss: 0.5738 - binary_accurac - ETA: 4:48 - l - ETA:  - ETA: 4:32 - loss: - ETA: 4:28 - loss: 0.5738 - binary_accuracy: 0.7 - ETA: 4:28 - loss: 0.5738  -  - ETA: 3:46 - loss: 0.5737 - binary_accuracy: 0.726 - ET - ETA: 3:41 - loss: 0.5737 - binary_ac - ETA: 3:39 -  - ETA: 3:34 - loss: 0.5737 - binary_ac - E - ETA: 2:50 - loss: 0.5736 - binary_accuracy: 0. - ETA: 2:24 - loss: 0.5735 - binary_accuracy: 0. - ETA: 2:23 - -
Epoch 3/4
5047/5047 [==============================] - 550s 109ms/step - loss: 0.5698 - binary_accuracy: 0.7398
Epoch 4/4
5047/5047 [==============================] - 546s 108ms/step - loss: 0.5698 - binary_accuracy: 0.7398 10:00 - loss:  - ETA: 4:31 - lo - ETA: 4:27 - loss: 0.5700 - binary_accu - - ETA: 4:07 - loss: 0.5699 - binary_accura - ETA: 3:59 - ETA: 3:55 - loss - ETA:  - ETA: 3:39 - loss: 0.5701 - binary_accuracy - ETA: 3:38 - loss: 0.5701 - binary_ac - ETA: 3: - ETA: 3:31 - loss: 0.5701 - binary_acc - ETA: 3:30 - loss: 0.5702 - bin - ETA: 3:27 - loss: 0 - ETA:  - ETA: 2:53 - lo - E - ETA: 2:37 - l - ETA: 2:32 - loss:  - ETA: 2:28 - loss: 0.5703 - binary_accuracy: 0. - ETA: 2:28 - loss: 0.5703 - binary_accura - ETA: 2:27 - loss: 0.57 - ETA: 2:23 - loss: 0.5702 - b - ETA: 2:21 -  - E - ETA: 54s - loss: 0.5698 - binary_a - ETA: 4s - loss: 0.5
"""

Epoch 1/4
636/636 [==============================] - 81s 127ms/step - loss: 5.9605e-08 - acc: 0.49961s - loss: 5.9636e-08 
Epoch 2/4
636/636 [==============================] - 90s 140ms/step - loss: 5.9605e-08 - acc: 0.4995 ETA: 4s -
Epoch 3/4
636/636 [==============================] - 73s 114ms/step - loss: 5.9605e-08 - acc: 0.4995
Epoch 4/4
636/636 [==============================] - 79s 124ms/step - loss: 5.9605e-08 - acc: 0.49978s - loss: 5.9809e-08 - ETA - ETA: 2s - loss: 5.9697e


'\nEpoch 1/4\n   2/5047 [..............................] - ETA: 3:09 - loss: 0.7847 - binary_accuracy: 0.515 - ETA: 9:20 - loss: 0.7903 - binary_accuracy: 0.5039WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0129s vs `on_train_batch_end` time: 0.2059s). Check your callbacks.\n5047/5047 [==============================] - 548s 109ms/step - loss: 0.5881 - binary_accuracy: 0.7075 8:59 - ETA: 8:3 - ETA: 8:01 - - ETA: 7:50 - loss: 0.6312  - ETA: 7:35 - loss: 0.6251 - bin - ETA: 7:13 - loss: 0.6196 - binary_acc - ETA: 7:11 - loss: 0.6192 -  - ETA: 7:08 - loss: 0.6188 -  - ETA: 7:04 - loss: 0.6183 - binary_accu - ETA: 7:04 - loss: 0.6179 - binary_a - ETA: 7:01 - loss: 0.6174 -  - ETA: 6:52 - los - ETA: 6:48 - loss: 0.6148 - binary_accuracy: 0. - ETA: 6:47 - loss: 0.6146 - - ETA: 6:45 - loss: 0.6 - ETA: 6:41  - ETA: 6:31 - loss: 0.6121 - binary_accuracy: 0.6 - ETA: 6:30 - loss: 0. - ETA: 6:26 - los - ETA: 6:02 - loss: 0.608 - ETA: 5:58

In [14]:
embedding_model = keras.Model(inputs=x_inp[0::2], outputs=x_out[0])

from stellargraph.mapper import GraphSAGENodeGenerator

node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_features_df.index)
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

print('Sample:', embeddings_wv['crooked.com'][:10])

80/80 [==============================] - 5s 57ms/step
Sample: [-0.015975967049598694, -0.004238066729158163, -0.029331792145967484, -0.04764040559530258, -0.028530171141028404, 0.030409863218665123, -0.03388119488954544, -0.017634505406022072, 0.0049674599431455135, 0.05562857538461685]


# Export embeddings as feature

In [15]:
export_model_as_feature(embeddings_wv, 'graph_sage_audience_overlap_level_1')

'/Users/panayot/Documents/News-Media-Peers/data/acl2020/features/graph_sage_audience_overlap_level_1.json'

In [17]:
run_experiment(features="graph_sage_audience_overlap_level_1", normalize_features=False)

+------+---------------------+---------------+--------------------+-------------------------------------+
| task | classification_mode | type_training | normalize_features |               features              |
+------+---------------------+---------------+--------------------+-------------------------------------+
| fact |  single classifier  |    combine    |       False        | graph_sage_audience_overlap_level_1 |
+------+---------------------+---------------+--------------------+-------------------------------------+


01-10 00:43:35 train        INFO     Start training...
01-10 00:43:35 train        INFO     Fold: 0
01-10 00:43:44 train        INFO     Fold: 1
01-10 00:43:51 train        INFO     Fold: 2
01-10 00:43:58 train        INFO     Fold: 3
01-10 00:44:04 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+-------------------------------------+-------------------+------------------+--------------------+--------------------+
| task | classification_mode | type_training | normalize_features |               features              |      Macro-F1     |     Accuracy     |  Flip error-rate   |        MAE         |
+------+---------------------+---------------+--------------------+-------------------------------------+-------------------+------------------+--------------------+--------------------+
| fact |  single classifier  |    combine    |       False        | graph_sage_audience_overlap_level_1 | 28.22290703646636 | 52.9685681024447 | 16.530849825378347 | 0.6356228172293364 |
+------+---------------------+---------------+--------------------+-------------------------------------+-------------------+------------------+--------------------+--------------------+
